In [25]:
import pandas as pd
import os

# Initialize an empty list to store DataFrames from each file
df_list = []

for filename in os.listdir("../dataset"):
    if filename.endswith('.csv'):
        try:
            # Read each CSV file into a DataFrame
            df = pd.read_csv(os.path.join("../dataset", filename))

            if not df.empty:
                # Append the DataFrame to the list
                df_list.append(df)
        except pd.errors.EmptyDataError:
            # skip empty files
            print(f"Warning: Empty file - {filename}")

data = pd.concat(df_list, ignore_index=True)

In [26]:
data

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt9114286,Black Panther: Wakanda Forever,2022,PG-13,161 min,"Action, Adventure, Drama",6.9,The people of Wakanda fight to protect their h...,Ryan Coogler,/name/nm3363032/,"Letitia Wright, \nLupita Nyong'o, \nDanai Guri...","/name/nm4004793/,/name/nm2143282/,/name/nm1775...",204835.0,NaN
1,tt1630029,Avatar: The Way of Water,2022,PG-13,192 min,"Action, Adventure, Fantasy",7.8,Jake Sully lives with his newfound family form...,James Cameron,/name/nm0000116/,"Sam Worthington, \nZoe Saldana, \nSigourney We...","/name/nm0941777/,/name/nm0757855/,/name/nm0000...",295119.0,NaN
2,tt5884796,Plane,2023,R,107 min,"Action, Thriller",6.5,A pilot finds himself caught in a war zone aft...,Jean-François Richet,/name/nm0724938/,"Gerard Butler, \nMike Colter, \nTony Goldwyn, ...","/name/nm0124930/,/name/nm1591496/,/name/nm0001...",26220.0,NaN
3,tt6710474,Everything Everywhere All at Once,2022,R,139 min,"Action, Adventure, Comedy",8.0,A middle-aged Chinese immigrant is swept up in...,"Dan Kwan, \nDaniel Scheinert",/name/nm3453283/,"Michelle Yeoh, \nStephanie Hsu, \nJamie Lee Cu...","/name/nm3215397/,/name/nm0000706/,/name/nm3513...",327858.0,NaN
4,tt5433140,Fast X,2023,NaN,NaN,"Action, Crime, Mystery",NaN,Dom Toretto and his family are targeted by the...,Louis Leterrier,/name/nm0504642/,"Vin Diesel, \nJordana Brewster, \nTyrese Gibso...","/name/nm0004874/,/name/nm0108287/,/name/nm0879...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368295,tt26698087,Janissary,NaN,NaN,NaN,"Drama, History, War",NaN,A young man is captured and brainwashed to fig...,NaN,NaN,NaN,NaN,NaN,NaN
368296,tt26690183,World War 1 project,NaN,NaN,NaN,War,NaN,Add a Plot,"Dominic Higgins, \nIan Higgins",/name/nm3836407/,NaN,/name/nm3838864/,NaN,NaN
368297,tt26695367,74,NaN,NaN,NaN,War,NaN,"74 centres on the lives of four women, and the...",NaN,NaN,NaN,NaN,NaN,NaN
368298,tt26698193,Pandora Palace,2019,NaN,NaN,"Action, Adventure, Drama",NaN,After Rajmahone Dictator Alexander Pandora lau...,Evan Kascinde,/name/nm11925490/,"Muhammad Al Abasiri, \nBrai Andujar, \nDon Bla...","/name/nm11799756/,/name/nm8613643/,/name/nm145...",NaN,NaN
